In [1]:
import torch
import torchvision

from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader, random_split


import os
import json
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image

### MNIST

In [3]:
train_dataset = torchvision.datasets.MNIST(root=r"C:\Users\voron\IT\pytorch", train=True, download=True)
test_dataset = torchvision.datasets.MNIST(root=r"C:\Users\voron\IT\pytorch", train=False, download=True)

100%|██████████████████████████████████████████████████████████████████████████████| 9.91M/9.91M [00:10<00:00, 990kB/s]
100%|██████████████████████████████████████████████████████████████████████████████| 28.9k/28.9k [00:00<00:00, 179kB/s]
100%|██████████████████████████████████████████████████████████████████████████████| 1.65M/1.65M [00:02<00:00, 600kB/s]
100%|██████████████████████████████████████████████████████████████████████████████| 4.54k/4.54k [00:00<00:00, 587kB/s]


In [2]:
import struct
import sys

from array import array
from os import path


In [3]:
def read(dataset):
    if dataset == "training":
        path_img = rf"C:\Users\voron\IT\pytorch\mnist\raw\train-images-idx3-ubyte"
        path_lbl = rf"C:\Users\voron\IT\pytorch\mnist\raw\train-labels-idx1-ubyte"
    elif dataset == "testing":
        path_img = rf"C:\Users\voron\IT\pytorch\mnist\raw\t10k-images-idx3-ubyte"
        path_lbl = rf"C:\Users\voron\IT\pytorch\mnist\raw\t10k-labels-idx1-ubyte"
    else:
        raise ValueError("Датасета нету")

    with open(path_lbl, "rb") as f_label:
        _, size = struct.unpack(">II", f_label.read(8))
        lbl = array("b", f_label.read())

    with open(path_img, "rb") as f_img:
        _, size, rows, cols = struct.unpack(">IIII", f_img.read(16))
        img = array("B", f_img.read())

    return lbl, img, size, rows, cols

def write_dataset(labels, data, size, cols, rows, output_dir):
    classes = {i: f"class_{i}" for i in range(10)}

    output_dirs = [
        path.join(output_dir, classes[i]) for i in range(10)
    ]

    for dir in output_dirs:
        if not path.exists(dir):
            os.makedirs(dir)

    for(i, label) in enumerate(labels):
        output_filename = path.join(output_dirs[label], str(i) + ".jpg")
        # print("writing" + output_filename)

        with open(output_filename, "wb") as h:
            data_i = [
                data[(i*rows*cols + j*cols) : (i*rows*cols + (j+1)*cols)] for j in range(rows)
            ]
            data_array = np.asarray(data_i)

            im = Image.fromarray(data_array)
            im.save(output_filename)

output_path = r"C:\Users\voron\IT\pytorch\data"
for dataset in ["training", "testing"]:
    write_dataset(*read(dataset), path.join(output_path, dataset))
        

### для регрессии

In [36]:
if not os.path.isdir(r"C:\Users\voron\IT\pytorch\dataset"):
    os.mkdir(r"C:\Users\voron\IT\pytorch\dataset")
img = np.random.randint(0, 50, [100000, 64, 64], dtype = np.uint8)
square = np.random.randint(100, 200, [100000, 15, 15], dtype = np.uint8)

cords = np.empty([100000, 2])

data = {}

for i in range(img.shape[0]):
    x = np.random.randint(20, 44)
    y = np.random.randint(20, 44)

    img[i, (y-7):(y+8), (y-7):(y+8)] = square[i]
    cords[i] = [y, x]

    name_img = f"img_{i}.jpeg"
    path_img = os.path.join(r"C:\Users\voron\IT\pytorch\dataset", name_img)

    image = Image.fromarray(img[i])
    image.save(path_img)

    data[name_img] = [y, x]

with open("dataset/cords.json", "w") as f:
    json.dump(data, f, indent=2)